<a href="https://colab.research.google.com/github/abiUni/colab/blob/main/Code_suggestions_codeT5_training_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download libraries



In [ ]:
# Install necessary libraries
!pip install datasets
!pip install transformers
!pip install evaluate

import torch
# choose device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# chrf dependencies
!pip install sacrebleu

# CodeBleu dependencies
!pip install tree-sitter
!git clone https://huggingface.co/spaces/giulio98/codebleu.git


# To get around a bug in Colab, see https://github.com/googlecolab/colabtools/issues/3409
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 33.4 MB/s eta 0:00

In [ ]:
# Login to Huggingface Hub
from huggingface_hub import login

login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Preprocess the data

In [ ]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from transformers import RobertaTokenizer

#use_auth_token=True if dataset is set to private
dataset = load_dataset('arefm/suggestions_small_py',)

print(dataset)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/arefm___csv/arefm--suggestions_small_py-a72fe9db3d0de0ad/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 10243
    })
    test: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 1281
    })
    validation: Dataset({
        features: ['id', 'original', 'modified'],
        num_rows: 1280
    })
})


In [ ]:
# loading the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

# setting the model's hyperparameters
max_source_length = 512
max_target_length = 512
# task_prefix helps exploite the knowledge learned by model
task_prefix = "Refine: "

# encoding the training data (inputs, target)
def process_data_to_model_inputs(batch):
  orgnl_code = batch['original'] # original code
  mdfd_code = batch['modified'] # the modified/suggested code 

  # encode the original code sequence
  inputs = [task_prefix + code for code in orgnl_code] 
   
  model_inputs = tokenizer(inputs, max_length=max_source_length, padding="max_length", truncation=True)
  # add data to the same device as the model
  # model_inputs = tokenizer(text, return_tensors="pt").input_ids.to(device)

  # encode the modified code sequence
  labels = tokenizer(mdfd_code, max_length=max_target_length, padding="max_length", truncation=True).input_ids


  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  # length of tokens
  batch["modified_code_len"] = [len(tokenizer(code).input_ids) for code in mdfd_code]
  batch["orig_code_len"] = [len(tokenizer(code).input_ids) for code in inputs]

  return model_inputs


# Prepare the training validation, and test data in batches

train =  dataset['train']
validation = dataset['validation']
test = dataset['test']

tokenized_train = train.map(process_data_to_model_inputs, batched=True, remove_columns=train.column_names)
tokenized_validation = validation.map(process_data_to_model_inputs, batched=True, remove_columns=validation.column_names)
tokenized_test = test.map(process_data_to_model_inputs, batched=True, remove_columns=test.column_names)

# Filter out longer code snippets

filtered_tokenized_train = tokenized_train.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length) 
filtered_tokenized_validation = tokenized_validation.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length)
filtered_tokenized_test = tokenized_test.filter(lambda db: db['modified_code_len'] <= max_source_length and db['orig_code_len'] <= max_source_length)

# Print how much data we kept after filtering in %
train_len = len(filtered_tokenized_train) / len(tokenized_train) * 100
validation_len = len(filtered_tokenized_validation) / len(tokenized_validation) * 100
test_len = len(filtered_tokenized_test) / len(tokenized_test) * 100

print(f'After removing code sequences that are longer than 512 tokens, we kept: Train: {train_len}%, Validation: {validation_len}%, Test: {test_len}%.')

Map:   0%|          | 0/10243 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10243 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1280 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1281 [00:00<?, ? examples/s]

After removing code sequences that are longer than 512 tokens, we kept: Train: 59.58215366591819%, Validation: 63.28125%, Test: 61.28024980483997%.


# Fine-tune using HuggingFace Trainer

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import GenerationConfig


# Pre-trained model initialization
if device == 'cuda':
  model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base').cuda()
else:
  model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

model.generation_config.max_length = 512

# Trainings hyperparameters
batch_size = 8
model_name = 'refine_suggestions_codet5-base' # custom name
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    generation_max_length = 512 # important to generate plausable code seq
)

# Pad the inputs in the batches to max_length
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Compute the predictions metrics

In [ ]:
import numpy as np
import evaluate
from codebleu.calc_code_bleu import calculate


def postprocess_code(preds, labels):
    preds = [pred.strip() for pred in preds] # ['pred one','pred two',' pred three']
    labels = [[label.strip()] for label in labels] # [['reference one'], ['reference two'], ['reference three']]

    return preds, labels

# we use 3 metrics: BLEU scores, accuracy (exactly match) and CodeBLEU.
# https://github.com/microsoft/CodeXGLUE/tree/main/Code-Code/code-refinement

# NOTE: CodeBLEU currently works on Linux machines only due to dependency from languages .so
# Check https://huggingface.co/docs/evaluate/transformers_integrations
def compute_metrics(eval_preds):
  # Load metrics
  # Check whether exact match can be used for code
  # https://huggingface.co/spaces/evaluate-metric/chrf
  exact_match = evaluate.load("chrf")
  # https://huggingface.co/spaces/evaluate-metric/bleu
  bleu = evaluate.load("bleu")
  results = {}

  preds, labels = eval_preds
  if isinstance(preds, tuple):
    preds = eval_preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


  # https://huggingface.co/spaces/giulio98/codebleu/blob/main/README.md
  codebleu = calculate(references=[decoded_labels] , predictions=decoded_preds , language="python", alpha=0.25, beta=0.25, gamma=0.25,
                         theta=0.25)
  results.update(codebleu)

  # Some simple post-processing (for CodeBleu we dont need post-processing)
  decoded_preds, decoded_labels = postprocess_code(decoded_preds, decoded_labels)

  #collapsed_labels_list = [item for sublist in decoded_labels for item in sublist]
  results.update(exact_match.compute(predictions=decoded_preds, references=decoded_labels))
  results.update(bleu.compute(predictions=decoded_preds, references=decoded_labels))

  return  results

# Note an optimizer is already defined for us
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=filtered_tokenized_train,
    eval_dataset=filtered_tokenized_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start finetuning the model

In [ ]:
trainer.train()

# Save the model locally
!mkdir t5_model
trainer.save_model("./t5_model/")
!zip -r /content/fine_tuned_model.zip /content/t5_model

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Ngram Match Score,Weighted Ngram Match Score,Syntax Match Score,Dataflow Match Score,Code Bleu Score,Score,Char Order,Word Order,Beta,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,0.213000,0.107062,0.935709,0.937275,0.932945,0.906224,0.928038,97.127519,6,0,2,0.963073,"[0.9770682023818258, 0.9666261166611371, 0.9584630013831259, 0.9509190651723325]",0.999847,0.999847,163572,163597
2,0.093700,0.102713,0.936624,0.937508,0.934342,0.908195,0.929167,97.211057,6,0,2,0.963478,"[0.9797510556539539, 0.9694442562469588, 0.9612996675931141, 0.953732448254624]",0.997380,0.997384,163169,163597
3,0.065900,0.103507,0.935917,0.936611,0.932616,0.905844,0.927747,97.162982,6,0,2,0.962808,"[0.9798643726930482, 0.9692884079761406, 0.9609410802977569, 0.9532378130757357]",0.996921,0.996925,163094,163597
4,0.047300,0.106128,0.937101,0.937882,0.933663,0.909613,0.929565,97.251146,6,0,2,0.963946,"[0.98032161986591, 0.9699379172723017, 0.9616784480730901, 0.9540511620962524]",0.997411,0.997414,163174,163597
5,0.039000,0.109883,0.936580,0.937399,0.933489,0.907019,0.928622,97.267628,6,0,2,0.964058,"[0.9805392156862746, 0.9699242564197302, 0.961548458967694, 0.9538160104497108]",0.997570,0.997573,163200,163597


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Trainer is attempting to log a value of "[0.9770682023818258, 0.9666261166611371, 0.9584630013831259, 0.9509190651723325]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9797510556539539, 0.9694442562469588, 0.9612996675931141, 0.953732448254624]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9798643726930482, 0.9692884079761406, 0.9609410802977569, 0.9532378130757357]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.98032161986591, 0.9699379172723017, 0.9616784480730901, 0.9540511620962524]" of type <class 'list'> for key "eval/

  adding: content/t5_model/ (stored 0%)
  adding: content/t5_model/tokenizer_config.json (deflated 77%)
  adding: content/t5_model/vocab.json (deflated 67%)
  adding: content/t5_model/generation_config.json (deflated 35%)
  adding: content/t5_model/special_tokens_map.json (deflated 97%)
  adding: content/t5_model/merges.txt (deflated 54%)
  adding: content/t5_model/training_args.bin (deflated 48%)
  adding: content/t5_model/config.json (deflated 62%)
  adding: content/t5_model/pytorch_model.bin (deflated 7%)


# Save model to GCS bucket

In [ ]:
# Login to GCS account
!gcloud auth login
!gcloud auth application-default login

In [ ]:
# Enter project ID
project_id = ''
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# Upload the pre-trained model to cloud

# To create new bucket use: !gsutil mb gs://{bucket_name}
bucket_name = 'bucket_testrun_sugg'

# Copy the file to our bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /content/fine_tuned_model.zip gs://{bucket_name}/

# Test the model performance

In [ ]:
# Run a test on the test dataset
trainer.predict(test_dataset= filtered_tokenized_test)

PredictionOutput(predictions=array([[  0,   1, 536, ...,   0,   0,   0],
       [  0,   1, 536, ...,   0,   0,   0],
       [  0,   1, 536, ...,   0,   0,   0],
       ...,
       [  0,   1, 536, ...,   0,   0,   0],
       [  0,   1, 536, ...,   0,   0,   0],
       [  0,   1, 536, ...,   0,   0,   0]]), label_ids=array([[   1,  536, 1842, ..., -100, -100, -100],
       [   1,  536,  336, ..., -100, -100, -100],
       [   1,  536, 1842, ..., -100, -100, -100],
       ...,
       [   1,  536,  752, ..., -100, -100, -100],
       [   1,  536, 2774, ..., -100, -100, -100],
       [   1,  536, 1109, ..., -100, -100, -100]]), metrics={'test_loss': 0.11283846199512482, 'test_ngram_match_score': 0.936925980559505, 'test_weighted_ngram_match_score': 0.9382355416006646, 'test_syntax_match_score': 0.9278280641917006, 'test_dataflow_match_score': 0.9209393190335698, 'test_code_bleu_score': 0.9309822263463601, 'test_score': 97.19541525250418, 'test_char_order': 6, 'test_word_order': 0, 'test_bet

# Load the finetuned model and generate output

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

model = T5ForConditionalGeneration.from_pretrained('arefm/refine_suggestions_codet5-base')

input_ids = tokenizer('''def print_hi():
    print(hi)''', return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))